In [ ]:
import pandas as pd
import sqlite3
import re
from datetime import datetime

# File paths
CUSTOMERS_CSV = "data/sources/customers.csv"
TRANSACTIONS_CSV = "data/sources/transactions.csv"
PRODUCTS_CSV = "data/sources/products.csv"
DB_NAME = "retail_data.db"

# Helper functions
def is_valid_email(email):
    return re.match(r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$", str(email))

def is_valid_date(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def clean_customers(df):
    df = df.dropna(subset=["email"])
    df = df[df["email"].apply(is_valid_email)]
    return df

def clean_transactions(df):
    df = df.drop_duplicates()
    df = df[df["transaction_date"].apply(is_valid_date)]
    return df

def clean_products(df):
    df["category"] = df["category"].str.strip().str.lower().str.capitalize()
    return df

def load_to_db(df, table_name, conn):
    df.to_sql(table_name, conn, if_exists='replace', index=False)


In [ ]:
customers = pd.read_csv(CUSTOMERS_CSV)

In [ ]:
customers.head()

In [ ]:
def main():
    # Extract
    customers = pd.read_csv(CUSTOMERS_CSV)
    transactions = pd.read_csv(TRANSACTIONS_CSV)
    products = pd.read_csv(PRODUCTS_CSV)
    
    # Transform
    customers = clean_customers(customers)
    transactions = clean_transactions(transactions)
    products = clean_products(products)
    
    # Load
    conn = sqlite3.connect(DB_NAME)
    load_to_db(customers, "customers", conn)
    load_to_db(transactions, "transactions", conn)
    load_to_db(products, "products", conn)
    
    # Create customer_revenue table
    aggregation_query = """
        CREATE TABLE IF NOT EXISTS customer_revenue AS
        SELECT c.customer_id, SUM(t.amount) AS total_amount
        FROM transactions t
        JOIN customers c ON t.customer_id = c.customer_id
        GROUP BY c.customer_id
    """
    conn.execute(aggregation_query)
    conn.commit()
    conn.close()